# Финальный проект. Декомпозиция

## Описание проекта

Датасет содержит данные о событиях, совершенных в мобильном приложении "Ненужные вещи". В нем пользователи продают свои ненужные вещи, размещая их на доске объявлений.
В датасете содержатся данные пользователей, впервые совершивших действия в
приложении после 7 октября 2019 года.

Колонки в /datasets/mobile_sources.csv :
- userId — идентификатор пользователя,
- source — источник, с которого пользователь установил приложение.
<br>

Колонки в /datasets/mobile_dataset.csv :
- event.time — время совершения,
- user.id — идентификатор пользователя,
- event.name — действие пользователя.
<br>

Виды действий:
- advert_open — открыл карточки объявления,
- photos_show — просмотрел фотографий в объявлении,
- tips_show — увидел рекомендованные объявления,
- tips_click — кликнул по рекомендованному объявлению,
- contacts_show и show_contacts — посмотрел номер телефона,
- contacts_call — позвонил по номеру из объявления,
- map — открыл карту объявлений,
- search_1 — search_7 — разные действия, связанные с поиском по сайту,
- favorites_add — добавил объявление в избранное.

## Задачи проекта

1. Проанализировать связь целевого события — просмотра контактов — и других действий пользователей.
2. Оценить, какие действия чаще совершают те пользователи, которые просматривают контакты.

## Цели

Анализ поведения пользователей приложения "Ненужные вещи" для увеличения конверсии в целевое действие, что в свою очередь увеличит вовлеченость

## Структура проекта




- __Знакомство с данными__
- __Предобработка данных__ 
    - Проверка дубликатов 
    - Проверка пропусков 
    - Типы данных
    - Названия столбцов
- __Иследовательский анализ данных__
    - Время проведеное в приложении
    - Retention Rate
- __Основные вопросы иследования__
    - В разрезе сессий отобрать сценарии, которые приводят к просмотру контактов
    - Строим воронки по основным сценариям в разрезе уникальных пользователей
    - Различия длитильности сессии с просмотром номера и без
    - Рассчитываем относительную частоту событий для тех кто смотрел контакты и тех кто не смотрел
    


- __Проверка гипотиз__
    - Одни пользователи совершают действия tips_show и tips_click, другие – только tips_show. Проверить гипотезу, что конверсия в просмотр контактов различается у этих двух групп.
    - Различается ли конверсия в целевое действие пользователей, которые добавляют объявление в избраное, от тех кто не добавляет.

## Знакомство с данными

Импортируем необходимые библиотеки

In [1]:
import pandas as pd

Далее загружаем датасеты и смотрим по пять строк каждого из них

In [2]:
dataset = pd.read_csv('https://code.s3.yandex.net/datasets/mobile_dataset.csv')
sources = pd.read_csv('https://code.s3.yandex.net/datasets/mobile_sources.csv')

In [3]:
dataset

,event.time,event.name,user.id
0,2019-10-07 00:00:00.431357,advert_open,020292ab-89bc-4156-9acf-68bc2783f894
1,2019-10-07 00:00:01.236320,tips_show,020292ab-89bc-4156-9acf-68bc2783f894
2,2019-10-07 00:00:02.245341,tips_show,cf7eda61-9349-469f-ac27-e5b6f5ec475c
3,2019-10-07 00:00:07.039334,tips_show,020292ab-89bc-4156-9acf-68bc2783f894
4,2019-10-07 00:00:56.319813,advert_open,cf7eda61-9349-469f-ac27-e5b6f5ec475c
...,...,...,...
74192,2019-11-03 23:53:29.534986,tips_show,28fccdf4-7b9e-42f5-bc73-439a265f20e9
74193,2019-11-03 23:54:00.407086,tips_show,28fccdf4-7b9e-42f5-bc73-439a265f20e9
74194,2019-11-03 23:56:57.041825,search_1,20850c8f-4135-4059-b13b-198d3ac59902
74195,2019-11-03 23:57:06.232189,tips_show,28fccdf4-7b9e-42f5-bc73-439a265f20e9


In [4]:
sources

,userId,source
0,020292ab-89bc-4156-9acf-68bc2783f894,other
1,cf7eda61-9349-469f-ac27-e5b6f5ec475c,yandex
2,8c356c42-3ba9-4cb6-80b8-3f868d0192c3,yandex
3,d9b06b47-0f36-419b-bbb0-3533e582a6cb,other
4,f32e1e2a-3027-4693-b793-b7b3ff274439,google
...,...,...
4288,b86fe56e-f2de-4f8a-b192-cd89a37ecd41,yandex
4289,424c0ae1-3ea3-4f1e-a814-6bac73e48ab1,yandex
4290,437a4cd4-9ba9-457f-8614-d142bc48fbeb,yandex
4291,c10055f0-0b47-477a-869e-d391b31fdf8f,yandex


Смотрим основную информацию 

In [5]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74197 entries, 0 to 74196
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   event.time  74197 non-null  object
 1   event.name  74197 non-null  object
 2   user.id     74197 non-null  object
dtypes: object(3)
memory usage: 1.7+ MB


In [6]:
sources.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4293 entries, 0 to 4292
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   userId  4293 non-null   object
 1   source  4293 non-null   object
dtypes: object(2)
memory usage: 67.2+ KB


### Вывод

## Предобработка данных

### Проверка дубликатов

Проверим на полные дубликаты

In [7]:
dataset.duplicated().sum()

0

И вторую таблицу тоже

In [8]:
sources.duplicated().sum()

0

Полные дубликаты отсутствуют в обеих таблицах. В столбце event_name у нас есть такие действия как contacts_show и show_contacts. Так как это одно и тоже, объединим их в одно целое.

In [9]:
dataset['event.name'] = dataset['event.name'].replace('show_contacts','contacts_show')

## Проверка пропусков

In [10]:
dataset.isna().sum()

event.time    0
event.name    0
user.id       0
dtype: int64

In [11]:
sources.isna().sum()

userId    0
source    0
dtype: int64

Не в одной из таблиц пропусков нет.

## Типы данных

У нас есть столбец с датой и веременем совершения пользователем того или иного действия. Столбец имеет тип данных object. И так как данные в столбце event.time до милисикунд, предлогаю их округлить до секунд. Такая точность нам вряд ли понадобится.

In [12]:
dataset['event.time'] = pd.to_datetime(dataset['event.time']).dt.round("s")

## Названия столбцов

Приводим название столбцов в snake case

In [13]:
dataset.columns=['event_time','event_name','user_id']

In [14]:
dataset

,event_time,event_name,user_id
0,2019-10-07 00:00:00,advert_open,020292ab-89bc-4156-9acf-68bc2783f894
1,2019-10-07 00:00:01,tips_show,020292ab-89bc-4156-9acf-68bc2783f894
2,2019-10-07 00:00:02,tips_show,cf7eda61-9349-469f-ac27-e5b6f5ec475c
3,2019-10-07 00:00:07,tips_show,020292ab-89bc-4156-9acf-68bc2783f894
4,2019-10-07 00:00:56,advert_open,cf7eda61-9349-469f-ac27-e5b6f5ec475c
...,...,...,...
74192,2019-11-03 23:53:30,tips_show,28fccdf4-7b9e-42f5-bc73-439a265f20e9
74193,2019-11-03 23:54:00,tips_show,28fccdf4-7b9e-42f5-bc73-439a265f20e9
74194,2019-11-03 23:56:57,search_1,20850c8f-4135-4059-b13b-198d3ac59902
74195,2019-11-03 23:57:06,tips_show,28fccdf4-7b9e-42f5-bc73-439a265f20e9


In [15]:
sources = sources.rename(columns={'userId': 'user_id'})

In [16]:
sources

,user_id,source
0,020292ab-89bc-4156-9acf-68bc2783f894,other
1,cf7eda61-9349-469f-ac27-e5b6f5ec475c,yandex
2,8c356c42-3ba9-4cb6-80b8-3f868d0192c3,yandex
3,d9b06b47-0f36-419b-bbb0-3533e582a6cb,other
4,f32e1e2a-3027-4693-b793-b7b3ff274439,google
...,...,...
4288,b86fe56e-f2de-4f8a-b192-cd89a37ecd41,yandex
4289,424c0ae1-3ea3-4f1e-a814-6bac73e48ab1,yandex
4290,437a4cd4-9ba9-457f-8614-d142bc48fbeb,yandex
4291,c10055f0-0b47-477a-869e-d391b31fdf8f,yandex


### Объединение таблиц

Для удобства работы объеденим две таблицы в одну

In [17]:
df = dataset.merge(sources,on='user_id',how='left')
df

,event_time,event_name,user_id,source
0,2019-10-07 00:00:00,advert_open,020292ab-89bc-4156-9acf-68bc2783f894,other
1,2019-10-07 00:00:01,tips_show,020292ab-89bc-4156-9acf-68bc2783f894,other
2,2019-10-07 00:00:02,tips_show,cf7eda61-9349-469f-ac27-e5b6f5ec475c,yandex
3,2019-10-07 00:00:07,tips_show,020292ab-89bc-4156-9acf-68bc2783f894,other
4,2019-10-07 00:00:56,advert_open,cf7eda61-9349-469f-ac27-e5b6f5ec475c,yandex
...,...,...,...,...
74192,2019-11-03 23:53:30,tips_show,28fccdf4-7b9e-42f5-bc73-439a265f20e9,google
74193,2019-11-03 23:54:00,tips_show,28fccdf4-7b9e-42f5-bc73-439a265f20e9,google
74194,2019-11-03 23:56:57,search_1,20850c8f-4135-4059-b13b-198d3ac59902,google
74195,2019-11-03 23:57:06,tips_show,28fccdf4-7b9e-42f5-bc73-439a265f20e9,google


### Вывод

## Иследовательский анализ

### Время проведеное в приложении

Для начала посмотрим сколько всего времяни провели пользователи в нашем приложении за все время. И в первую очередь выясним за какой времяной промежуток у нас имеются данные.

In [18]:
df['event_time'].min()

Timestamp('2019-10-07 00:00:00')

In [19]:
df['event_time'].max()

Timestamp('2019-11-03 23:58:13')

Итак с 07.10.2019 по 03.11.2019. 28 дней или 4 недели.

### Retention Rate

## Основные вопросы иследования

### 1

### 2

### 3

### 4

## Проверка гипотез 

### Первая

### Вторая

## Вывод